# ECS759P Lab 2: Tree search algorithms (BFS and DFS)

In this lab, we are going to implement and compare two of our basic search algorithms, namely Breadth First Search (**BFS**) and Depth First Search (**DFS**) on a real-world problem. 

But first, a quick tutorial on graph visualisation.

## 0. How to represent a graph (a tutorial)

Most of the search algorithms we study need to be applied to problems that can be modelled as graphs, where the graph represents the state space of the problem.   

You might be familiar with modelling a "tree", which is a graph without any cycles. While modelling a tree may be relatively simple, there exists many ways to encode a graph.

We can for instance use nested dictionaries to represent a graph, as in the following example:

In [ ]:
simple_graph = {"node_A" : {"node_B" : 10, "node_C" : 75, "node_D" : 20}, 
                "node_B" : {"node_A" : 10, "node_C" : 35, "node_D" : 25}, 
                "node_C" : {"node_B" : 35, "node_A" : 75, "node_D" : 30}, 
                "node_D" : {"node_B" : 25, "node_A" : 20, "node_C" : 30}}

The above dictionary contains all the information required to represent a graph. Each *key* of the `simple_graph` dictionary corresponds to a *vertex* `v`, a.k.a. a *node*. The *value* of a vertex is itself a dictionary, containing the information about the *neighbours* of that vertex. For instance `node_A` is connected to `node_B`, `node_C` and `node_D` via *edges*, a.k.a., *links*,  having *step costs*, a.k.a. *weights* of 10, 75, and 20, respectively. 

An advantage of this representation is the ease of access to all the information. A disadvantage is that it is rather memory-inefficient (note the size of the dictionary to represent a graph of only 4 nodes and 6 links), as it contains a lot of redundant information. 


As a warm up, with a pen and paper or in your head, try to visualise the graph and then find the shortest path from `node_A` to `node_C`. For the (simple) example above, **once** you visualize the graph, it is quite straightforward to analyse it. However, this can quickly show its limitations. For instance, let's try to visualise the following example:

In [ ]:
more_complex_graph = {"vertex1" : {"vertex2" : 24, "vertex3" : 52, "vertex0" : 17}, 
                      "vertex2" : {"vertex0" : 14, "vertex3" : 64, "vertex5" : 11, "vertex1" : 24, "vertex4": 41}, 
                      "vertex0": {"vertex1" : 17, "vertex2" : 14, "vertex5" : 44}, 
                      "vertex3": {"vertex1" : 52, "vertex2" : 64}, 
                      "vertex4" : {"vertex2" : 41}, 
                      "vertex5": {"vertex2" : 11, "vertex0" : 44}}

It's quite difficult isn't it? 

That is why we are going to use a useful python library called **[NetworkX](https://networkx.github.io/documentation/stable/)**. This library provides some useful tools to build, analyse and display graphs. One of its classes allowing to encode a graph is called `Graph`. Although there exists once again a lot of ways to generate graphs with this library, we are going to go through the basics. Consider the following example:

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

dummy_nx_graph = nx.Graph()
dummy_nx_graph.add_node("Hello")
dummy_nx_graph.add_node("amigo.")

Here we just added two nodes (or vertices). Let's visualise them:

In [ ]:
nx.draw(dummy_nx_graph, with_labels=True, node_size=1500, font_weight='bold')
plt.show()

Well, two nodes alone, without any edge between them, ain't that interesting! So let's add a link: 

In [ ]:
dummy_nx_graph.add_edge("Hello", "amigo.")
nx.draw(dummy_nx_graph, with_labels=True, node_size=1500, font_weight='bold')
plt.show()

We can also make our graph grow just by adding edges (which will also create the necessary new nodes for us). 

We can add weights (step costs) to each of the existing edges, or alternatively, specify the weight when creating an edge:

In [ ]:
dummy_nx_graph["Hello"]["amigo."]['weight'] = 1 # adding a weight to an existing edge
dummy_nx_graph.add_edge("amigo.",  "Ready", weight=1.5) # adding a new edge and specifying its weight
nx.draw(dummy_nx_graph, with_labels=True, node_size=1500, font_weight='bold') 
plt.show()

Adding nodes or edges one by one would be too painful for big graphs. Instead, we can add multiple edges at once from an iterable container:

In [ ]:
dummy_nx_graph.add_nodes_from(["Ready", "for"])
dummy_nx_graph.add_weighted_edges_from([("Ready", "for", 4.2), 
                                        ("for", "today", 9.8), 
                                        ("today", "?", 7)])
nx.draw(dummy_nx_graph, with_labels=True, node_size=1500, font_weight='bold')
plt.show()

**Q. By filling the gaps below, create and visualise the equivalent of `simple_graph` that we specified using nested dictionaries at the beginning of this lab.**

In [ ]:
simple_nx_graph = nx.Graph()

for node, connected_elem in simple_graph.items():
    # TO DO
    for connected_node, weight in connected_elem.items():
        simple_nx_graph.add_edge(node, connected_node, weight = weight)

nx.draw(simple_nx_graph, with_labels=True, node_size=1500, font_weight='bold')
plt.show()

This is more convenient than reading and understanding a graph from a structure like a nested dictionary. 

However, the visualisation still leaves something to be desired: the weights are not appearing! Let's solve this by first drawing only the vertices and then the weighted edges.

In [ ]:
def show_weighted_graph(networkx_graph, node_size, font_size, fig_size):
  # Allocate the given fig_size in order to have space for each node
  plt.figure(num=None, figsize=fig_size, dpi=80)
  plt.axis('off')
  # Compute the position of each vertex in order to display it nicely
  nodes_position = nx.spring_layout(networkx_graph) 
  # You can change the different layouts depending on your graph
  # Extract the weights corresponding to each edge in the graph
  edges_weights  = nx.get_edge_attributes(networkx_graph,'weight')
  # Draw the nodes (you can change the color)
  nx.draw_networkx_nodes(networkx_graph, nodes_position, node_size=node_size,  
                         node_color = ["orange"]*networkx_graph.number_of_nodes())
  # Draw only the edges
  nx.draw_networkx_edges(networkx_graph, nodes_position, 
                         edgelist=list(networkx_graph.edges), width=2)
  # Add the weights
  nx.draw_networkx_edge_labels(networkx_graph, nodes_position, 
                               edge_labels = edges_weights)
  # Add the labels of the nodes
  nx.draw_networkx_labels(networkx_graph, nodes_position, font_size=font_size, 
                          font_family='sans-serif')
  plt.axis('off')
  plt.show()

show_weighted_graph(dummy_nx_graph, 1500, 15, (10,5))

Even though we can now display graphs nicely, creating large graphs is still painful. That is why we are going to use another nice feature of `networkX` that allows creating a graph from a dictionary.

In [ ]:
the_more_complex_graph = nx.Graph(more_complex_graph)
show_weighted_graph(the_more_complex_graph, 2500, 12, (10,7))

The weights contained inside the dictionary are not displayed. That's because the library expects a specific format like the following:

`{"nodeA" : {"nodeB" : {"weight" : 10}}}`

Let's try:

In [ ]:
show_weighted_graph(nx.Graph({"nodeA" : {"nodeB" : {"weight" : 10}}}), 1000, 10, (5,5))

**Q. By filling the gaps below, reformat `more_complex_graph` in order to display it properly (with the weights).**

In [ ]:
more_complex_dict = dict()

for node, connected_elem in more_complex_graph.items():
  # TO DO

show_weighted_graph(nx.Graph(more_complex_dict), 2500, 12, (10,7))

Besides nice visualisation, `networkX` provides a lot of functions that are going to make our life easier. It can for instance give us the number of nodes (`<graph>.number_of_nodes()`), number of edges (`<graph>.number_of_edges()`), the name and values of nodes and edges (`list(<graph>.edges())` and `list(<graph>.nodes())`), the neighbours of a given node (`list(<graph>.neighbors(<node_name>))`), adjacencies and so on. 

For more information you can have a look at the documentation [here](https://networkx.github.io/documentation/stable/tutorial.html#).

Now that we are a bit familiar with how to work with graphs using python, let's tackle some problems! 

## 1. Find your way out!

One of the main application of tree search algorithms is to find a path between an initial state to a goal state. There are many problems that can be modelled in this framework, but in our first example, we are simply going to find our way out of a given labyrinth.


Before proceeding with the rest of the exercises, **double check if the other auxiliary files are available in the same folder of this notebook**. The files include three data files with `json` extension and some (labyrinth) images.  If you don't remember how, refer to the previous lab manual, or just ask a demonstrator. 

### 1.1. First steps

In this subsection you are going to find a path between the entrance (top left corner) and the exit (bottom right corner) of a given labyrinth using **BFS** and **DFS** algorithms.

![Image of the first labyrinth](https://drive.google.com/uc?id=1h-IV8LNcAV2_c3IIIuOOJONjZCBz2bZw)

The graph corresponding to this labyrinth is provided in the file **`small_labyrinth.json`**. Let's load it:

In [ ]:
import json

def load_graph_from_file(filename):
  with open(filename) as labyrinth_file:
    dict_labyrinth = json.load(labyrinth_file)
    return nx.Graph(dict_labyrinth)

small_labyrinth = load_graph_from_file("small_labyrinth.json")
show_weighted_graph(small_labyrinth, 500, 10, (10,10))

The cells named `entrance` and `exit` are self-explanatory and are respectively equivalent to `C00` and `C44`. All the other cells are named `CXY` where `X` corresponds to the row and `Y` column of the cell in the image.


*NOTE: In this simple exercise, the arguably more difficult part is already done for us, which is to abstract the original problem (finding an exit path in a labyrinth) and cast is as a tree/graph search problem. This might not be the case in your coursework/exams, so take a moment to ponder about how this abstraction worked, and if you could have done it yoursef.*

### 1.2. Depth First Search (DFS)

**Q. By filling the gaps below, write two python functions (a non-recursive and a recursive one) each returning a solution: a list of nodes composing the path found by DFS from the starting node to the node representing the goal state (including both). Alternatively, you can also return a list composing the edges that represent a solution path. Add the functionality to reverse the order of the explored nodes at each level.**

*Hint: In your code, you can take advantage of the method `<graph>.neighbors(<node_name>)` to find which nodes are connected to another.*

### DFS - Non-recursive

In [ ]:
def construct_path_from_root(node, root):
    # TO DO
 
    return path_from_root


def my_depth_first_graph_search(nxobject, initial, goal, compute_exploration_cost=False, reverse=False):
    frontier = [{'label':initial, 'parent':None}]  
    explored = {initial}
    number_of_explored_nodes = 1 

    while frontier:
        # TO DO
    return None

You can now call your function with `small_labyrinth` (the example graph object we constructed before, representing our small labyrinth), `"entrance"` and `"exit"` as test arguments. Print the output list, and check if it should return `["entrance", "C10", "C11", "C21", "C31", "C32", "C22", "C23", "C33", "C34", "exit"]`.

In [ ]:
solution = my_depth_first_graph_search(small_labyrinth, 'entrance', 'exit')
construct_path_from_root(solution, 'entrance')

### DFS - Recursive

In [ ]:
def my_recursive_dfs_implementation(graph, origin, destination, already_visited = [], count=1, reverse=False):
  # If I reach destination, I finish right here, return list with the final place
  if origin == destination:
    return [origin], count+1
  # TO DO

  # If no node works, I return empty string which would mean dead end
  return [], count+1

In [ ]:
path, _ = my_recursive_dfs_implementation(small_labyrinth, 'entrance', 'exit')
print(path)

### 1.3. Breadth First Search (BFS)


**Q. By filling the gaps below, write two functions (an implementation with a queue and an implementation with a list) taking the same inputs as before but returning the list containing the nodes that represent the path found by BFS.**

### BFS with a Queue

In [ ]:
def my_breadth_first_graph_search(nxobject, initial, goal, compute_exploration_cost=False, reverse=False):
    
    if initial == goal: # just in case, because now we are checking the children
        return None
    
    number_of_explored_nodes = 1    
    frontier = [{'label':initial, 'parent':None}]  
    # FIFO queue implementation with a list is slow. For bigger problems, better to use deque.
    explored = {initial}
    
    while frontier:
        # TO DO
            
    return None

Calling your new function with the same input as before, (`small_labyrinth`, `"entrance"` and `"exit"`) you should find exactly the same output: `["entrance", "C10", "C11", "C21", "C31", "C32", "C22", "C23", "C33", "C34", "exit"]`


In [ ]:
solution = my_breadth_first_graph_search(small_labyrinth, 'entrance', 'exit')
construct_path_from_root(solution, 'entrance')

### BFS with a Search Tree

In [ ]:
def bfs_implementation(graph, origin, destination, counter = 0, reverse=False):
  # Add current place to already_visited
  next_already_visited = [origin]
  # List of existent paths (for now only origin)
  total_paths = [[origin]] 

  # Will perform exploration of all current paths
  while len(total_paths)!= 0: 
    new_total_paths = []
    # I check every single existing path for now
    for path in total_paths:
      # TO DO
 
    # At the end, I need to explore only these "new" paths, until I reach destination, or run out of possible valid paths
    total_paths = new_total_paths

  # If no more possible paths, means solution does not exist
  return [],-1

In [ ]:
bfs_path, _ = bfs_implementation(small_labyrinth, 'entrance', 'exit')
print(bfs_path)

### 1.4. A bit of analysis

You might wonder why we should implement both BFS and DFS if they output exactly the same path at the end. If it's not already clear, let's investigate a bit more: 

**Q. Modify your code so that it also returns the number of nodes visited along with the solution path. Then run the analysis below.** 

###  BFS and DFS - Variation 1

In [ ]:
solution_bfs = my_breadth_first_graph_search(small_labyrinth, 'entrance', 'exit', True)
solution_dfs = my_depth_first_graph_search(small_labyrinth, 'entrance', 'exit', True)

### BFS and DFS - Variation 2

In [ ]:
bfs_path, number_visited_bfs = bfs_implementation(small_labyrinth, 'entrance', 'exit')
print("Number of visited for BFS: {}".format(number_visited_bfs))
dfs_path, number_visited_dfs = my_recursive_dfs_implementation(small_labyrinth, 'entrance', 'exit')
print("Number of visited for DFS: {}".format(number_visited_dfs))

Now, for DFS, let's change the way you pick the next node. For instance, if the neighbours of a node are `[a, b, c]`, instead of visiting `a` first, let's visit `c` first.

### BFS and DFS - Variation 1

In [ ]:
solution_bfs2 = my_breadth_first_graph_search(small_labyrinth, 'entrance', 'exit', True, True)
solution_dfs_2 = my_depth_first_graph_search(small_labyrinth, 'entrance', 'exit', True, True)

### BFS and DFS - Variation 2

In [ ]:
bfs_path_i, number_visited_bfs_inversed = bfs_implementation(small_labyrinth, 'entrance', 'exit', reverse=True)
print("Number of visited for inversed BFS: {}".format(number_visited_bfs_inversed))
dfs_path_i, number_visited_dfs_inversed = my_recursive_dfs_implementation(small_labyrinth, 'entrance', 'exit', reverse=True)
print("Number of visited for inversed DFS: {}".format(number_visited_dfs_inversed))

If everything works well, the path returned should be exactly the same. But what about the number of cells you have visited? Can you do the same thing with BFS? Explain why.

### 1.5. BFS or DFS?


The last subsection showed that how we choose to explore in DFS will impact the efficiency of the search. So why not just using BFS then? Let's try to understand why. 


Let's consider a larger labyrinth. Use your two search functions that you implemented in order to find a path between `"entrance"` and `"exit"`. The graph you are going to work with corresponds to the following labyrinth:

![Image: a larger labyrinth example.](https://drive.google.com/uc?id=14KFgxnoM4P5UFSkb7Hth30YBR7QPAcRx)




In [ ]:
large_labyrinth = load_graph_from_file("large_labyrinth.json")
show_weighted_graph(large_labyrinth, 500, 10, (30, 30))

### Variation 1

In [ ]:
# You can call both your BFS and DFS function in this cell with large_labyrinth, "entrance" and "exit".

print('\nUsing BFS:\n'+'='*10)
solution_bfs = my_breadth_first_graph_search(large_labyrinth, 'entrance', 'exit', True)
print(construct_path_from_root(solution_bfs, 'entrance'))

print('\nUsing DFS:\n'+'='*10)
solution_dfs = my_depth_first_graph_search(large_labyrinth, 'entrance', 'exit', True)
print(construct_path_from_root(solution_dfs, 'entrance'))


### Variation 2

In [ ]:
print('\nUsing BFS:\n'+'='*10)
bfs_path, number_visited_bfs = bfs_implementation(large_labyrinth, 'entrance', 'exit')
print("Number of explorations: {}".format(number_visited_bfs))
print(bfs_path)

print('\nUsing DFS:\n'+'='*10)
dfs_path, number_visited_dfs = my_recursive_dfs_implementation(large_labyrinth, 'entrance', 'exit')
print("Number of explorations: {}".format(number_visited_dfs))
print(dfs_path)

By the way, both of your algorithms should output a list corresponding to the following solution:

![alt text](https://drive.google.com/uc?id=1qTM-WlnWaUJNELzwgfpgR6qVuk-KrW9S)

Let's now carry out the same analysis as before: how many nodes have been visited using DFS? using BFS? Can you come up with a "rule" or a "trend"?


### Variation 1

In [ ]:
print('\nUsing BFS:\n'+'='*10)
solution_inversed_bfs = my_breadth_first_graph_search(large_labyrinth, 'entrance', 'exit', True, True)

print('\nUsing DFS:\n'+'='*10)
solution_inversed_dfs = my_depth_first_graph_search(large_labyrinth, 'entrance', 'exit', True, True)

### Variation 2

In [ ]:
print('\nUsing BFS:\n'+'='*10)
bfs_path_i, number_visited_bfs_i = bfs_implementation(large_labyrinth, 'entrance', 'exit', reverse=True)
print("Number of explorations: {}".format(number_visited_bfs_i))

print('\nUsing DFS:\n'+'='*10)
dfs_path_i, number_visited_dfs_i = my_recursive_dfs_implementation(large_labyrinth, 'entrance', 'exit', reverse=True)
print("Number of explorations: {}".format(number_visited_dfs_i))

Note that in both scenarios, DFS visits less states. The difference is even more obvious with big labyrinth with a lot of walls. On one hand, BFS ensures you to get the optimal solution on non weighted graph and is less efficient on trees with an important branching factor. On the other hand, DFS is more efficient of such problems but might explore more solutions than BFS if the graph is very deep.

# 2. Limits of BFS and DFS


In the previous examples, except for the number of nodes visited before finding the solution path, we cannot intuitively evaluate whether the solution path is good or bad since there exists only one path (the solution is unique). 

In some other problems, just finding a solution path is not enough or interesting, and we rather want to find the best possible solution (the one with the lowest cost). 

For instance, let's say for travelling from city A to city B, most of the travellers would prefer taking the shortest path. Let's see if BFS and DFS can help in such scenarios.

In [ ]:
uk_cities = load_graph_from_file("UK_cities.json")
show_weighted_graph(uk_cities, 2500, 10, (50, 50))

Run both your DFS and BFS on this graph in order to find a path between London and Aberdeen. The arguments should be `"uk_cities", "london"` and `"aberdeen"`. 



### Variation 1

In [ ]:
# Code to run and compute the cost of the path found by DFS and BFS
solution_bfs_ = my_breadth_first_graph_search(uk_cities, 'london', 'aberdeen', True)
print("Path found by BFS: {}".format(construct_path_from_root(solution_bfs_, 'london')))
solution_dfs_ = my_depth_first_graph_search(uk_cities, 'london', 'aberdeen', True)
print("Path found by DFS: {}".format(construct_path_from_root(solution_dfs_, 'london')))

In [ ]:
# Code to run and compute the cost of the path found by DFS and BFS
solution_bfs_i = my_breadth_first_graph_search(uk_cities, 'london', 'aberdeen', True, True)
print("Path found by inversed BFS: {}".format(construct_path_from_root(solution_bfs_i, 'london')))
solution_dfs_i = my_depth_first_graph_search(uk_cities, 'london', 'aberdeen', True, True)
print("Path found by inversed DFS: {}".format(construct_path_from_root(solution_dfs_i, 'london')))

### Variation 2

In [ ]:
bfs_path, number_visited_bfs = bfs_implementation(uk_cities, 'london', 'aberdeen')
print("Number of explorations: {}".format(number_visited_bfs))
print("Path found by BFS: {}".format(bfs_path))
dfs_path, number_visited_dfs = my_recursive_dfs_implementation(uk_cities, 'london', 'aberdeen')
print("Number of explorations: {}".format(number_visited_dfs))
print("Path found by DFS: {}".format(dfs_path))

In [ ]:
bfs_path_i, number_visited_bfs_i = bfs_implementation(uk_cities, 'london', 'aberdeen', reverse=True)
print("Number of explorations: {}".format(number_visited_bfs_i))
print("Path found by BFS: {}".format(bfs_path_i))
dfs_path_i, number_visited_dfs_i = my_recursive_dfs_implementation(uk_cities, 'london', 'aberdeen', reverse=True)
print("Number of explorations: {}".format(number_visited_dfs_i))
print("Path found by DFS: {}".format(dfs_path_i))

**Q. By filling the gaps below, compute the cost of the path found by each algorithm. Can you guarantee that you have the shortest path? Explain why.**

In [ ]:
def compute_path_cost(graph, path):
  cost = 0
  # TO DO
  
  return cost

### Variation 1

For this implementation, we only have two generated paths.

In [ ]:
path_one = construct_path_from_root(solution_dfs_, 'london')
path_two = construct_path_from_root(solution_dfs_i, 'london')

first_path_cost = compute_path_cost(uk_cities, path_one)
second_path_cost = compute_path_cost(uk_cities, path_two)

print("Cost for path {}: {}".format(path_one, first_path_cost))
print("Cost for path {}: {}".format(path_two, second_path_cost))

###  Variation 2

For this implementation, we have four different paths.

In [ ]:
bfs_cost_path = compute_path_cost(uk_cities, bfs_path)
dfs_cost_path = compute_path_cost(uk_cities, dfs_path)
inversed_bfs_cost_path = compute_path_cost(uk_cities, bfs_path_i)
inversed_dfs_cost_path = compute_path_cost(uk_cities, dfs_path_i)

print("Cost for BFS path {}: {}".format(bfs_path, bfs_cost_path))
print("Cost for DFS path {}: {}".format(dfs_path, dfs_cost_path))
print("Cost for inversed BFS path {}: {}".format(bfs_path_i, inversed_bfs_cost_path))
print("Cost for inversed DFS path {}: {}".format(dfs_path_i, inversed_dfs_cost_path))